In [1]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
import importlib
from astropy.cosmology import FlatLambdaCDM

from utils_orbs.orb_paths import SetupPaths
import utils_orbs.readsubfHDF5Py3 as readSub
from utils_orbs.read_group_cats import ReadCats
from utils_orbs.merger_trees import TraceMergerTree
from utils_orbs.readMergerHistoryFiles import FindHistory
from utils_orbs.vectorCorrection import vectorCorrection as vector

paths = SetupPaths()

imported h5py...
modules seem to have loaded?


In [2]:
plt.show();
plt.rcParams.update({'font.size':16,"xtick.direction":"in","ytick.direction":"in", 
                     "xtick.top":True, "ytick.right":True,"text.usetex":False,
                     "xtick.labelsize":18,"ytick.labelsize":18})

# Definitions for this notebook:

In [3]:
z_directory = {"33":"z=2","40":"z=1.5","50":"z=1"}

## Load in data from hydro catalog

In [52]:
# load in z=1.5 cat
catpath = paths.path_tnghydro
keysel_default =  [
            'GroupPos','Group_M_TopHat200', 
            'Group_R_TopHat200','GroupNsubs',
            'GroupFirstSub',
            'SubhaloGrNr','SubhaloMass',
            'SubhaloPos','SubhaloVel' ]
    
catalog = readSub.subfind_catalog(
            basedir=catpath,
            snapnum=50,
            keysel=keysel_default
            )

In [53]:
catalog.redshift

0.9972942257819404

In [54]:
catalog.SubhaloGrNr[345349]

2781

## Load in data from orbit data

In [4]:
orbit_dict = {}
for snap in [33,40,50]:
    f = h5py.File(f"{paths.path_data}orbitdata_{snap}.hdf5",'r')
    orbit_dict[str(snap)] = {}
    for key, val in f.items():
        orbit_dict[str(snap)][key] = np.array(val)
    f.close()

f = h5py.File(f"{paths.path_data}snapshot_data.hdf5",'r')
snap_dict = {}
for key, val in f.items():
    snap_dict[key] = np.array(val)
f.close()

In [39]:
loc = np.where(snap_dict['Snapshot'][::-1][:-1]==40)[0]


In [6]:
print(f"Subfind ID 1: {orbit_dict['50']['SubfindID1'][0]}")
print(f"Subfind ID 2: {orbit_dict['50']['SubfindID2'][0]}")

Subfind ID 1: 296198.0
Subfind ID 2: 296197.0


In [28]:
orbit_dict['50']['Separations'][i][loc[0]]

29.659290018986198

In [21]:
orbit_dict['40']['Separations'][4]

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         3.87299324,  15.54514329,  28.72411657,  15.52381958,
        48.77615373,  69.82232054,  83.13017474,  87.59695578,
        86.47259912,  76.58478689,  64.12880754,  52.69403326,
        59.09816758,  78.10851464, 104.75594497, 128.66122267,
       145.8184515 , 170.63988386, 188.38308619, 202.64539896,
       215.10186892, 225.89459571, 243.063789  , 256.93761605,
       276.30978543, 289.89131736, 305.78816854, 323.29356808,
       331.81403432, 351.32876001, 368.17057697, 383.43

In [25]:
orbit_dict['50']['SubfindID1'][13]

250969.0

In [24]:
print("comov:",orbit_dict['40']['Separations'][13][loc[0]]/scale*little_h)
print("phys:",orbit_dict['40']['Separations'][13][loc[0]])

comov: 12.566809998335831
phys: 7.420614111801494


In [40]:
scale = 1/(2.5)
little_h  = 0.6774
for i in range(100):
    print(f"{i}: {orbit_dict['40']['Separations'][i][loc[0]]/scale*little_h} ckpc/h") 

0: 314.19962831622627 ckpc/h
1: 614.2084940020025 ckpc/h
2: 522.1321301294241 ckpc/h
3: 261.18252113441145 ckpc/h
4: 547.4976575368576 ckpc/h
5: 402.9521303989837 ckpc/h
6: 280.01690064800914 ckpc/h
7: 307.9563836962059 ckpc/h
8: 254.49572602327353 ckpc/h
9: 413.37960946768385 ckpc/h
10: 286.90529593815256 ckpc/h
11: 295.33979007006724 ckpc/h
12: 321.67559765341696 ckpc/h
13: 12.566809998335831 ckpc/h
14: 280.390542754227 ckpc/h
15: 289.59804681311226 ckpc/h
16: 339.05284964223966 ckpc/h
17: 277.66129067162296 ckpc/h
18: 346.97554180680373 ckpc/h
19: 356.89211663004914 ckpc/h
20: 393.20371737765186 ckpc/h
21: 281.11878356792147 ckpc/h
22: 262.6011454022076 ckpc/h
23: 301.880480200599 ckpc/h
24: 296.52736192212825 ckpc/h
25: 344.2068507399659 ckpc/h
26: 367.2314971852575 ckpc/h
27: 245.08438710151668 ckpc/h
28: 225.15473819166158 ckpc/h
29: 298.91473159715053 ckpc/h
30: 211.0262503837661 ckpc/h
31: 286.203294309895 ckpc/h
32: 487.16259326742573 ckpc/h
33: 326.8696146949095 ckpc/h
34: 20

In [51]:
print("z=1, snapshot 50")
print("---")
for i in [12,20,49,69,72,80,99]:
    scale = 1/2
    loc = np.where(snap_dict['Snapshot'][::-1][:-1]==50)[0]
    print(f"- Subhalo {orbit_dict['50']['SubfindID1'][i]:g}/{orbit_dict['50']['SubfindID2'][i]:g} with comov:{orbit_dict['50']['Separations'][i][loc[0]]/scale*little_h:.2f} ckpc/h  and phys: {orbit_dict['50']['Separations'][i][loc[0]]:.2f} ckpc/h")

print("z=1.5, snapshot 40")
print("---")
for i in [1,13,76,79,94]:
    scale = 1/2.5
    loc = np.where(snap_dict['Snapshot'][::-1][:-1]==40)[0]
    print(f"- Subhalo {orbit_dict['40']['SubfindID1'][i]:g} and {orbit_dict['40']['SubfindID2'][i]:g} with comov:{orbit_dict['40']['Separations'][i][loc[0]]/scale*little_h:.2f} ckpc/h  and phys: {orbit_dict['40']['Separations'][i][loc[0]]:.2f} ckpc/h")
          

z=1, snapshot 50
---
- Subhalo 345349/345352 with comov:28.72 ckpc/h  and phys: 21.20 ckpc/h
- Subhalo 354245/354247 with comov:69.77 ckpc/h  and phys: 51.50 ckpc/h
- Subhalo 364526/364528 with comov:36.83 ckpc/h  and phys: 27.18 ckpc/h
- Subhalo 375812/375814 with comov:43.55 ckpc/h  and phys: 32.15 ckpc/h
- Subhalo 377390/377392 with comov:37.03 ckpc/h  and phys: 27.33 ckpc/h
- Subhalo 379964/379966 with comov:45.35 ckpc/h  and phys: 33.47 ckpc/h
- Subhalo 386982/386978 with comov:40.18 ckpc/h  and phys: 29.66 ckpc/h
z=1.5, snapshot 40
---
- Subhalo 223269 and 223270 with comov:614.21 ckpc/h  and phys: 362.69 ckpc/h
- Subhalo 250969 and 250973 with comov:12.57 ckpc/h  and phys: 7.42 ckpc/h
- Subhalo 285676 and 285678 with comov:46.32 ckpc/h  and phys: 27.35 ckpc/h
- Subhalo 286766 and 286769 with comov:93.78 ckpc/h  and phys: 55.38 ckpc/h
- Subhalo 294517 and 294519 with comov:58.33 ckpc/h  and phys: 34.45 ckpc/h


In [7]:
loc = np.where(snap_dict['Snapshot'][::-1][:-1]==40)[0]
print(f"Separation: {orbit_dict['50']['Separations'][0][loc[0]]} kpc") 
print(f"Velocity: {orbit_dict['50']['Velocities'][0][loc[0]]} km/s")

Separation: 518.9960620880878 kpc
Velocity: 519.6427001953125 km/s


List of close pairs that could make for cool plots: 

z=1, snapshot 50
---
- Subhalo 345349 with comov:28.72 ckpc/h  and phys: 21.20 ckpc/h
- Subhalo 354245 with comov:69.77 ckpc/h  and phys: 51.50 ckpc/h
- Subhalo 364526 with comov:36.83 ckpc/h  and phys: 27.18 ckpc/h
- Subhalo 375812 with comov:43.55 ckpc/h  and phys: 32.15 ckpc/h
- Subhalo 377390 with comov:37.03 ckpc/h  and phys: 27.33 ckpc/h
- Subhalo 379964 with comov:45.35 ckpc/h  and phys: 33.47 ckpc/h
- Subhalo 386982 with comov:40.18 ckpc/h  and phys: 29.66 ckpc/h

z=1.5, snapshot 40
---
- Subhalo 250969 with comov:12.57 ckpc/h  and phys: 7.42 ckpc/h
- Subhalo 285676 with comov:46.32 ckpc/h  and phys: 27.35 ckpc/h
- Subhalo 286766 with comov:93.78 ckpc/h  and phys: 55.38 ckpc/h
- Subhalo 294517 with comov:58.33 ckpc/h  and phys: 34.45 ckpc/h


In [50]:
6*scale/little_h

3.54295837023915